# Notebook for training the Neural Network

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import json
import os

## Check for GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Define model

In [3]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 10)  # No softmax here; CrossEntropyLoss expects raw logits
        )

    def forward(self, x):
        return self.model(x)

## Load MNIST data

In [4]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=1000, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 501kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.58MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.97MB/s]


## Training

In [8]:
model = SimpleMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 15
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

# Evaluate accuracy
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Epoch 1/15 - Loss: 0.6129
Epoch 2/15 - Loss: 0.2686
Epoch 3/15 - Loss: 0.2243
Epoch 4/15 - Loss: 0.1992
Epoch 5/15 - Loss: 0.1820
Epoch 6/15 - Loss: 0.1712
Epoch 7/15 - Loss: 0.1614
Epoch 8/15 - Loss: 0.1543
Epoch 9/15 - Loss: 0.1468
Epoch 10/15 - Loss: 0.1401
Epoch 11/15 - Loss: 0.1353
Epoch 12/15 - Loss: 0.1306
Epoch 13/15 - Loss: 0.1259
Epoch 14/15 - Loss: 0.1228
Epoch 15/15 - Loss: 0.1202
Test Accuracy: 95.35%


## Export weights

In [9]:
def export_model_to_json(model, filename='model_weights.json'):
    model_weights = {}
    for name, param in model.named_parameters():
        model_weights[name] = param.detach().cpu().numpy().tolist()

    with open(filename, 'w') as f:
        json.dump(model_weights, f)

export_model_to_json(model)
print("Model weights exported to model_weights.json")

Model weights exported to model_weights.json


# Get raw 28x28 test arrays

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist

# Load MNIST test data
(_, _), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to range [0, 1]
x_test = x_test.astype(np.float32) / 255.0

# Get one sample for digits 0, 1, and 7
targets = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
selected = {}

for digit in targets:
    idx = np.where(y_test == digit)[0][0]
    selected[digit] = x_test[idx]

# Function to format a numpy 28x28 array into JS array string
def format_as_js_array(array):
    return "[\n" + ",\n".join(
        ["  [" + ", ".join(f"{v:.2f}" for v in row) + "]" for row in array]
    ) + "\n]"

# Output JS-formatted arrays
for digit, array in selected.items():
    print(f"const digit{digit} = {format_as_js_array(array)};\n")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
const digit0 = [
  [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
  [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
  [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
  [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
  [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.04, 0.59, 0.99, 0.79, 0.12, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
  [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0